---
title: "Fama-French 5-factor Model Implementation"
author: Sangwon Woo
date: September 17, 2024
format:
  html:
    code-fold: false
jupyter: python3
image: "ff5f.png"
categories: [factor model, code, analysis]
---

# This is a post with Fama-French 5 Factor Model 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.width', 160)
pd.set_option('display.precision', 18)

In [2]:
def get_size(x, m1, m2, m3, m4):
    if x<=m1:
        return 'S1'
    elif x>m1 and x<=m2:
        return 'S2'
    elif x>m2 and x<=m3:
        return 'S3'
    elif x>m3 and x<=m4:
        return 'S4'
    elif x>m4:
        return 'S5'
    
def get_beme(x, m1, m2, m3, m4):
    if x<=m1:
        return 'B1'
    elif x>m1 and x<=m2:
        return 'B2'
    elif x>m2 and x<=m3:
        return 'B3'
    elif x>m3 and x<=m4:
        return 'B4'
    elif x>m4:
        return 'B5'

In [3]:
info = pd.read_csv("info.csv")
del info['Name'], info['주기'], info['Symbol'], info['결산월']
info.columns = ['year', 'industry', 'ticker', 'exchcd']
info = info[info['exchcd'] == '유가증권시장']
info = info[info['industry'] != 'K']
info = info.sort_values(by=['ticker', 'year']).reset_index(drop=True)[['ticker', 'year', 'exchcd']]
info = info[~(info['ticker'].isin(['A008080', 'A005560']))]

In [4]:
fdf_c = ['ticker', 'name', 'year', 't-1장부가치']
ret_c = ['ticker', 'year', '시가총액']

In [5]:
fdf = pd.read_csv("finance.csv")
fdf.columns = ['ticker', 'name', '결산월', 'year', '보통주자본금', '자본잉여금', '이익잉여금', '자기주식', '이연법인세부채', '총자본', '우선주자본금', '이연법인세자산', '영업이익', '총자산']

for c in fdf.columns.tolist():
    if c in ['ticker', 'name', '결산월', 'year']: continue
    fdf.loc[:, c] = fdf[c]*1000
    
fdf = fdf.sort_values(by=['ticker', 'year'])
fdf = fdf[fdf['보통주자본금'].notna()].reset_index(drop=True)
fdf['t-1장부가치'] = fdf.groupby(by='ticker')[['보통주자본금', '자본잉여금', '이익잉여금', '자기주식', '이연법인세부채']].shift(1).sum(axis=1)
fdf['결산월'] = fdf['결산월'].astype('int8')
fdf['t-1영업이익'] = fdf.groupby('ticker')['영업이익'].shift(1)
fdf['OP'] = fdf['t-1영업이익'] / fdf['t-1장부가치']
fdf['t-1총자산'] = fdf.groupby(by='ticker')['총자산'].shift(1)
fdf['t-2총자산'] = fdf.groupby(by='ticker')['총자산'].shift(2)
fdf['INV'] = (fdf['t-1총자산']-fdf['t-2총자산']) / fdf['t-1총자산']
fdf = fdf[['ticker', 'year', 't-1장부가치', 'OP', 'INV']]

In [6]:
df = pd.read_csv("월별주가.csv")
df = df.melt(id_vars=['Symbol', 'Name']).rename(columns={'Symbol':'ticker', "variable":'date', 'value':'price'}).sort_values(by=['ticker', 'date'])
df['price-12m'] = df.groupby(by='ticker')['price'].shift(12)
df['price-1m'] = df.groupby(by='ticker')['price'].shift(1)
df['mom'] = (df['price-1m'] - df['price-12m']) / df['price-12m'] * 100
df['월수익률'] = (df['price'] - df['price-1m']) / df['price'] * 100
df = df[np.where(df[['price', 'price-12m', 'price-1m', 'mom', '월수익률']].isna().sum(axis=1)==5, False, True)].reset_index(drop=True)
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df = df[['date', 'year', 'month', 'ticker', '월수익률', 'price', 'price-1m', 'mom']]

In [7]:
rmrf = pd.read_csv("rmrf.csv")
rmrf['Date'] = pd.to_datetime(rmrf['Date'], format='%Y%m')
rmrf['year'], rmrf['month'] = rmrf['Date'].dt.year, rmrf['Date'].dt.month

In [8]:
ret = pd.read_csv("return.csv")
cols = ret.columns.tolist()
n_cols = [cols[0], cols[4]] + cols[6:]
ret = ret[n_cols]
ret = ret.rename(columns={"Symbol":"ticker", "Item Name ":"item"})
size, rts = ret['item'].unique().tolist()
ret.loc[ret['item'] == size, "item"] = '시가총액'
ret.loc[ret["item"] == rts, "item"] = '월수익률'
ret = pd.melt(ret, id_vars=["ticker", "item"], var_name='date')
ret = pd.pivot(ret, index=['ticker', 'date'], columns='item').reset_index()
ret.columns = ['ticker', 'date', '시가총액', '월수익률']
ret = ret[['ticker', 'date', '시가총액']]
ret['date'], ret['시가총액'] = pd.to_datetime(ret['date']), ret['시가총액'] * 1000000
ret['year'], ret['month'] = ret['date'].dt.year, ret['date'].dt.month
ret = ret[ret['시가총액'].notna()].reset_index(drop=True)
ret = pd.merge(ret, ret[ ret['month'] == 6][ret_c], how='left', on=['ticker', 'year'])
ret = ret.rename(columns={'시가총액_x':'t시총', '시가총액_y':'SIZE'})
ret = ret.merge(info, how='left', on=['ticker', 'year'])
ret = ret[ret['exchcd'].notna()].reset_index(drop=True)
ret = ret[['year', 'month', 'ticker', 't시총', 'SIZE']]

In [9]:
rdf = df.merge(ret, how='left', on=['year', 'month', 'ticker'])

In [10]:
ret = pd.DataFrame()
for c, df in rdf.groupby(by='ticker'):
    # print(c, end=' ')
    df['t-1시총'] = df['t시총'].shift(12)
    df = pd.merge(df, df[ df['month'] == 6][['ticker', 'year', 't-1시총']], how='left', on=['ticker', 'year'])
    del df['t-1시총_x']
    df = df.rename(columns={'t-1시총_y':'t-1시총'})
    ret = pd.concat([ret, df])
ret = ret.reset_index(drop=True)

In [11]:
tdf = ret.merge(fdf, how='left', on=['year', 'ticker'])
tdf = tdf.assign(BEME=tdf['t-1장부가치'] / tdf['t-1시총'])
tdf = tdf.merge(rmrf, how='left', on=['year', 'month'])
tdf = tdf[['date', 'year', 'month', 'ticker', '월수익률', 'price', 'price-1m', 'mom', 't시총', 't-1시총','RM-RF', 'SIZE', 'BEME', 'OP', 'INV']]

In [12]:
df = pd.DataFrame()
for y in [2017, 2018, 2019, 2020, 2021, 2022]:
    ydf = tdf[(tdf['year']==y)&(tdf['month']==6)]
    ydf = ydf[(ydf['SIZE'].notna())&(ydf['BEME'].notna())&(ydf['OP'].notna())&(ydf['INV'].notna())]
    
    s1 = ydf['SIZE'].quantile(0.5)
    b1, b2 = ydf['BEME'].quantile([0.3, 0.7])
    o1, o2 = ydf['OP'].quantile([0.3, 0.7])
    i1, i2 = ydf['INV'].quantile([0.3, 0.7])
    
    ydf = ydf.assign(size=np.where(ydf['SIZE']<=s1, 'small', 'big'))
    ydf = ydf.assign(beme=np.where(ydf['BEME']<=b1, 'value', np.where(ydf['BEME']<=b2, 'neutral', 'growth')))
    ydf = ydf.assign(op=np.where(ydf['OP']<=o1, 'robust', np.where(ydf['OP']<=o2, 'neutral', 'weak')))
    ydf = ydf.assign(inv=np.where(ydf['INV']<=i1, 'conservative', np.where(ydf['INV']<=i2, 'neutral', 'aggressive')))
    ydf=  ydf[['ticker', 'size', 'beme', 'op', 'inv']]
    
    rdf = tdf[tdf['date'].between(f'{y}0701', f'{y+1}0630')].merge(ydf, how='left', on=['ticker'])
    rdf['port_year'] = str(y)
    df = pd.concat([df, rdf])

In [13]:
df = df[(df['size'].notna())&(df.beme.notna())&(df.op.notna())&(df.inv.notna())].reset_index(drop=True)
df = df[['port_year', 'ticker', 'year', 'month', '월수익률', 'size', 'beme', 'op', 'inv']]

In [14]:
mat = pd.DataFrame()

for c, tf in df.groupby(by='port_year'):
    tf['size-beme'] = tf['size']+' '+ tf['beme']
    tf['size-op'] = tf['size']+' '+ tf['op']
    tf['size-inv'] = tf['size']+' '+ tf['inv']
        
    sf = tf.groupby(by=['size-beme', 'month'])['월수익률'].mean()
    for m, zf in sf.groupby('month'):
        df.loc[(df['port_year']==c)&(df['month']==m), "SMB(B/M)"] = ((zf['small value'] + zf['small neutral'] + zf['small growth'])/3 - (zf['big value'] + zf['big neutral'] + zf['big growth'])/3).tolist()[0]
        df.loc[(df['port_year']==c)&(df['month']==m), "HML"] = ((zf['small value'] + zf['big value'])/2 - (zf['small growth'] + zf['big growth'])/2).tolist()[0]
    
    sf = tf.groupby(by=['size-op', 'month'])['월수익률'].mean()
    for m, zf in sf.groupby('month'):
        df.loc[(df['port_year']==c)&(df['month']==m), "SMB(OP)"] = ((zf['small robust'] + zf['small neutral'] + zf['small weak'])/3 - (zf['big robust'] + zf['big neutral'] + zf['big weak'])/3).tolist()[0]
        df.loc[(df['port_year']==c)&(df['month']==m), "RMW"] = ((zf['small robust'] + zf['big robust'])/2 - (zf['small weak'] + zf['big weak'])/2).tolist()[0]
        
    sf = tf.groupby(by=['size-inv', 'month'])['월수익률'].mean()
    for m, zf in sf.groupby('month'):
        df.loc[(df['port_year']==c)&(df['month']==m), "SMB(INV)"] = ((zf['small conservative'] + zf['small neutral'] + zf['small aggressive'])/3 - (zf['big conservative'] + zf['big neutral'] + zf['big aggressive'])/3).tolist()[0]
        df.loc[(df['port_year']==c)&(df['month']==m), "CMA"] = ((zf['small conservative'] + zf['big conservative'])/2 - (zf['small aggressive'] + zf['big aggressive'])/2).tolist()[0]
    
    df.loc[(df['port_year']==c), "SMB"] = (df['SMB(B/M)'] + df['SMB(OP)'] + df['SMB(INV)'])/3
df = df.groupby(by=['year', 'month'])[['SMB', 'HML', 'RMW', 'CMA']].mean().reset_index()
df = df.merge(rmrf[['year', 'month', 'RM-RF']], how='left', on=['year', 'month'])

In [15]:
for t, asdf in tdf.groupby(by=['year', 'month']):
    if t[0] < 2017: continue
    if t[0]==2017 and t[1] < 6: continue
    asdf = asdf[['year', 'month', '월수익률', 'price-1m', 'mom']]
    asdf=asdf.dropna().sort_values('mom')
    
    m1, m2 = asdf['mom'].quantile([0.3, 0.7])
    asdf.loc[asdf['mom']<=m1, "MOM"] = 'low'
    asdf.loc[asdf['mom']>m2, 'MOM'] = 'high'
    high, low = asdf.groupby('MOM')[['month', '월수익률']].mean()['월수익률'].tolist()
    df.loc[(df['year']==t[0])&(df['month']==t[1]), "MOM"] = high-low
    
    r1, r2 = asdf['price-1m'].quantile([0.3, 0.7])
    asdf.loc[asdf['price-1m']<=r1, "REV"] = 'low'
    asdf.loc[asdf['price-1m']>r2, 'REV'] = 'high'
    high, low = asdf.groupby('REV')[['month', '월수익률']].mean()['월수익률'].tolist()
    df.loc[(df['year']==t[0])&(df['month']==t[1]), "REV"] = low-high

In [16]:
df.to_csv('factor.csv', index=False)

In [17]:
# 5x5 portfolio 수익률
df = pd.DataFrame()
for y in [2017, 2018, 2019, 2020, 2021, 2022]:
    ydf = tdf[(tdf['year']==y)&(tdf['month']==6)]
    m1, m2, m3, m4 = ydf['SIZE'].quantile([0.2, 0.4, 0.6, 0.8])
    ydf = ydf.assign(size=ydf['SIZE'].apply(get_size, args=(m1, m2, m3, m4)))
    m1, m2, m3, m4 = ydf['BEME'].quantile([0.2, 0.4, 0.6, 0.8])
    ydf = ydf.assign(beme=ydf['BEME'].apply(get_beme, args=(m1, m2, m3, m4)))
    ydf.loc[:, 'portfolio'] = ydf['year'].astype("str") + ydf['size'] + ydf['beme']
    ydf = ydf[['ticker', 'portfolio']]
    
    rdf = tdf[tdf['date'].between(f'{y}0701', f'{y+1}0630')].merge(ydf, how='left', on=['ticker'])
    rdf = rdf.groupby("portfolio")[['월수익률']].mean().reset_index().sort_values('portfolio')
    rdf.loc[:, 'year'] = rdf['portfolio'].str[:4]
    rdf.loc[:, 'port'] = rdf['portfolio'].str[4:]
    rdf = rdf.pivot(columns='year', index='port', values='월수익률')
    df = pd.concat([df, rdf], axis=1)
df = df.T.mean().reset_index()
df['SIZE'] = df['port'].str[:2]
df['BE/ME'] = df['port'].str[2:]
df = df.pivot(columns='BE/ME', index='SIZE', values=0)

In [18]:
df.to_csv('5x5 portfolio.csv')